In [15]:
import cohere
import openai, random, os
from qdrant_client import QdrantClient
from qna import responding_openai
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI API
openai.api_key = os.environ['OPENAI_API_KEY']
# Cohere API
co = cohere.Client('4ECOTqDXJpIYhxMQhUZxY12PPSqvgtYFclJm4Gnz')
# Define QdrantClient CONGDOAN
os.environ['QDRANT_URL'] = "https://e8a5fbc5-cfa2-4203-8a6d-d5be737d36e6.us-east4-0.gcp.cloud.qdrant.io:6333"
os.environ['QDRANT_API_KEY'] = 'ZDM2poNcaFvsCtwAQLBWwTDJkfkXoTbB5mOd5C839TTKI_DdiIiWfQ'
os.environ['QDRANT_COLLECTION_NAME'] = 'context'

qdrant_client = QdrantClient(url=os.environ['QDRANT_URL'], api_key=os.environ['QDRANT_API_KEY'])


In [24]:
qdrant_client.count(collection_name=os.environ['QDRANT_COLLECTION_NAME'])

CountResult(count=2)

### RAG (Conventional)

In [16]:
# Openai Embedding a text to vector
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
def get_embedding_co(text):
    return co.embed(texts=[text], model="multilingual-22-12")

# Search Qdrant Docs
def search_qdrant(query, top_k=3, embed='openai'):
    if embed == 'cohere':
        query_vector = get_embedding_co(query).embeddings[0]
    else:
        query_vector = get_embedding(query)
    search_result = qdrant_client.search(
        collection_name=os.environ['QDRANT_COLLECTION_NAME'],
        query_vector=query_vector,
        limit=top_k
    )
    score_dict =  {result.payload['page_content']: result.score for result in search_result}
    return search_result, score_dict

In [18]:
# Querying
query = "Thị trường dầu thô đang diễn biến ra sao?"
# Get relevant documents
unranked_search_result, unranked_score_dict = search_qdrant(query, embed='cohere')

In [27]:
cont = unranked_search_result[0].payload['page_content']
score_ = unranked_search_result[1].score
len(cont)
print(score_)

0.7509419


In [28]:
unranked_score_dict.values()

dict_values([0.80444384, 0.7509419])

In [14]:
# Querying
query = "Thị trường dầu thô đang diễn biến ra sao?"
# Get relevant documents
unranked_search_result, unranked_score_dict = search_qdrant(query)
# Re-arrange the received documents
search_info = "\n\n".join(unranked_search_result[i].payload['page_content'] for i in range(len(unranked_search_result)))
# Generate response with OpenAI
results, chatgpt_response_time = responding_openai(query, search_info)
print(results)

Length of Prompt exceeds the limitation of LLM input. Task closed!


UnboundLocalError: cannot access local variable 'results' where it is not associated with a value